# Implement models with PyTorch to allow for GPU cuda compatability

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [5]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # tain the model
    num_epochs = 15
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [8]:
train_model("csvs/ml_X.csv", "built_models/torch_ml.pth")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
train_model("csvs/ou_X.csv", "built_models/torch_ou.pth")

Epoch 1/15, Loss: 0.6916184425354004
Epoch 2/15, Loss: 0.6932923197746277
Epoch 3/15, Loss: 0.6967695951461792
Epoch 4/15, Loss: 0.6956311464309692
Epoch 5/15, Loss: 0.6896898150444031
Epoch 6/15, Loss: 0.7005860805511475
Epoch 7/15, Loss: 0.685490071773529
Epoch 8/15, Loss: 0.6887855529785156
Epoch 9/15, Loss: 0.6978492736816406
Epoch 10/15, Loss: 0.6967654228210449
Epoch 11/15, Loss: 0.6930269002914429
Epoch 12/15, Loss: 0.7010118365287781
Epoch 13/15, Loss: 0.6905204057693481
Epoch 14/15, Loss: 0.7023320198059082
Epoch 15/15, Loss: 0.7006088495254517
Accuracy: 0.47809899255365745


In [ ]:
train_model("csvs/pl_X.csv", "built_models/torch_pl.pth")

Epoch 1/15, Loss: 0.7258080840110779
Epoch 2/15, Loss: 0.7723819613456726
Epoch 3/15, Loss: 0.5177258253097534
Epoch 4/15, Loss: 0.670660674571991
Epoch 5/15, Loss: 0.6865572929382324
Epoch 6/15, Loss: 0.6642751693725586
Epoch 7/15, Loss: 0.6472684741020203
Epoch 8/15, Loss: 0.6330243349075317
Epoch 9/15, Loss: 0.6574232578277588
Epoch 10/15, Loss: 0.7036466598510742
Epoch 11/15, Loss: 0.3860982060432434
Epoch 12/15, Loss: 0.6571404337882996
Epoch 13/15, Loss: 0.4709460139274597
Epoch 14/15, Loss: 0.32402750849723816
Epoch 15/15, Loss: 0.4484308660030365
Accuracy: 0.4921190893169877
